In [ ]:
mport threading
import time
import emoji
import re
import random
import requests
import pandas as pd
from bs4 import BeautifulSoup

headers = {'User-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0'}
section_url = 'https://icook.tw/categories?ref=icook-footer'

res = requests.get(section_url, headers=headers)
soup_section = BeautifulSoup(res.text,'html.parser')
tag_list = soup_section.select('a.categories-all-child-link')
section_ls = []
for tag in tag_list:
    section_ls.append(tag['href'])

your_section = section_ls[len(section_ls)//2:]
your_section=your_section[4:8]

def preprocess (x):
    x = emoji.get_emoji_regexp().sub(r'', x)
    x = re.sub(r'[［(（【{][^】)}）］]*[】)}）］]', '', x)
    x = x.strip().replace(" ", "")
    x = re.sub(r'[~\-。]', "", x) 
    return x

def job(x):
    #headers = {'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36'}
    domain = 'https://icook.tw'
    url = x
    page_index = 1
    c = 0
    full_url = domain + url
    df = pd.DataFrame(columns = ['Recipe Name','Author','Recipe URL','Recipe Picture','Servings','Time','Ingredients',\
                                 'Unit','Recipe Like','Publish Date','Views Number','Recipe Steps'])
    res = requests.get(full_url, headers=headers)
    soup = BeautifulSoup(res.text,'html.parser')
    section_number = url.split('/')[2]

    while(full_url != domain):
        print(f"URL: {full_url}")
        res = requests.get(full_url, headers=headers)
        soup = BeautifulSoup(res.text,'html.parser')
        food_list = soup.select('a.browse-recipe-touch-link')
        for food in food_list:
            recipe_url = 'https://icook.tw/' + food['href']
            res_recipe = requests.get(recipe_url,headers=headers)
            soup_recipe = BeautifulSoup(res_recipe.text,'html.parser')
            #recipe_section = soup_recipe.select('a.recipe-breadcrumb-item-link')[1].text.strip()
            recipe_title = "".join((soup_recipe.select('h1.title')[0].text).split())
            recipe_title = preprocess(recipe_title)
            recipe_author = soup_recipe.select('a.author-name-link')[0].text
            recipe_pic_url = soup_recipe.select("img.main-pic")[0]['src']
            
        
            if len(soup_recipe.select('span.stat-content')) == 2:
                recipe_like = 0
            else:
                if '萬' in soup_recipe.select('span.stat-content')[0].text:
                    recipe_like = int(soup_recipe.select('span.stat-content')[0].text[:-5].replace('.','')+"000")
                else:
                    recipe_like = int(soup_recipe.select('span.stat-content')[0].text[:-3].replace(',',''))
        
            if not soup_recipe.select('div.servings-info.info-block') and not soup_recipe.select('div.time-info.info-block'):
                recipe_servings = 0
                recipe_time = 0
            elif not soup_recipe.select('div.time-info.info-block'):
                recipe_servings = int(soup_recipe.select('span.num')[0].text)
                recipe_time = 0
            elif  not soup_recipe.select('div.servings-info.info-block'): 
                recipe_servings = 0
                recipe_time = int(soup_recipe.select('span.num')[0].text)
            else: 
                recipe_servings = int(soup_recipe.select('span.num')[0].text)
                recipe_time = int(soup_recipe.select('span.num')[1].text)

            recipe_ingredient = ""
            for ingredient in soup_recipe.select('.ingredient-search'):
                if ingredient.text == "請參考圖片說明":
                    continue
                else:
                    recipe_ingredient += ingredient.text + " " 
            recipe_ingredient = emoji.get_emoji_regexp().sub(r'', recipe_ingredient)
        
            recipe_unit = ""
            for unit in soup_recipe.select('div.ingredient-unit'):
                if unit.text == "内文有":
                    continue
                else:
                    recipe_unit += unit.text + " "

            recipe_steps = ""
            counter = 1
            for steps in soup_recipe.select('div.step-instruction-content'):
                recipe_steps += str(counter) + " " + steps.text + "\n"
                recipe_steps = emoji.get_emoji_regexp().sub(r'', recipe_steps)
                counter += 1
            
            publish_date = soup_recipe.select('span.meta-content')[0].text[:-3]
            views = soup_recipe.select('span.meta-content')[1].text[:-3]
            #print(views)
        
            if '萬' in views:
                views_number = views[:-2]
                views_number = int(views_number.replace('.','')+"000")
            else:
                views_number = int(views.replace(',',''))
        
        
            
            #print(recipe_section)    
            print(recipe_url)
            #print(recipe_title)
            #print(recipe_author)
            #print(recipe_pic_url)
            #print(recipe_like)
            #print(recipe_servings)
            #print(recipe_time)
            #print(recipe_ingredient)
            #print(recipe_unit)
            #print(recipe_steps)
            #print(publish_date)
            #print(views)
            
        
            ls = [recipe_title,recipe_author,recipe_url,recipe_pic_url,recipe_servings,recipe_time,recipe_ingredient,\
                  recipe_unit,recipe_like,publish_date,views_number,recipe_steps]
            df.loc[c] = ls
            c += 1
    
            time.sleep(random.randint(2, 5))
        
        pages = soup.select('a[rel="next"]')
    
        if len(pages) != 0:
            print(pages)
            page = pages[0]["href"]
            full_url = domain + page
            #time.sleep(random.randint(1, 3))
        else:
            full_url = domain
        
    df.to_csv(f'./{section_number}.csv', index=False ,encoding='utf-8-sig')


threads = list()
starttime = time.time()
#a = ['/categories/347','/categories/346']
for element in your_section:
    threads.append(threading.Thread(target=job, args=(element,)))
for thread in threads:    
    thread.start()
for thread in threads:
    thread.join()
print('Multithread took {} seconds'.format(time.time() - starttime))
